In [ ]:
import psi4
import forte

In [ ]:
def run_psi4_casscf(geom, basis, occ, reference='rhf'):
    """
    Run a Psi4 CASSCF.
    :param geom: a string for molecular geometry
    :param basis: a string for basis set
    :param occ: a map of {mo_space: occupation}, e.g., {'docc': [3,0,1,1]}
    :param reference: a string for the type of reference
    :return: a tuple of (casscf energy, psi4 Wavefunction)
    """
    
    if not all(space in occ for space in ['rdocc', 'active', 'docc']):
        raise ValueError(f'{occ} does not contain keys "rdocc", "docc", or "active".')
    
    psi4.core.clean()
    mol = psi4.geometry(geom)
    
    options_dict = {'basis': basis,
                    'reference': reference,
                    'scf_type': 'pk',
                    'maxiter': 150,
                    'e_convergence': 1.0e-8,
                    'd_convergence': 1.0e-6,
                    'docc': occ['docc'],
                    'restricted_docc': occ['rdocc'],
                    'active': occ['active'],
                    'mcscf_maxiter': 100,
                    'mcscf_e_convergence': 1.0e-8,
                    'mcscf_r_convergence': 1.0e-6,
                    'mcscf_diis_start': 20
                   }
    
    psi4.set_options(options_dict)
    
    psi4.core.set_output_file('output.dat', False)
    Ecas, wfn = psi4.energy('casscf', return_wfn=True)
    psi4.core.clean()
    return Ecas, wfn

In [ ]:
geom = """0 1
O
H 1 1.2
H 1 1.2 2 120.0
"""

basis = '6-31g'

occ = {'docc': [3,0,1,1], 'rdocc': [2,0,0,0], 'active': [2,0,1,2]}

In [ ]:
# Valence orbitals of water

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(6,4), dpi=150)
ax = plt.subplot(111)
img = mpimg.imread('water_valence_orbs.png')
ax.set_axis_off()
ax.imshow(img)

In [ ]:
Ecas, wfn = run_psi4_casscf(geom, basis, occ, 'rhf')
print(f"CASSCF energy: {Ecas:.12f} Eh")

In [ ]:
from forte import forte_options

def pre_dsrg(ref_wfn, mo_spaces, rdm_level=3):
    """
    Preparation step for DSRG: compute a CAS and its RDMs.
    :param ref_wfn: reference wave function from psi4
    :param mo_spaces: a dictionary {mo_space: occupation}, e.g., {'ACTIVE': [0,0,0,0]}
    :param rdm_level: max RDM to be computed
    :return: a tuple of (reference energy, MOSpaceInfo, ForteIntegrals, RDMs)
    """
    
    forte.startup()
    
    # pass Psi4 options to Forte
    options = psi4.core.get_options()
    options.set_current_module('FORTE')
    forte_options.update_psi_options(options)
    
    # create a MOSpaceInfo object
    mo_space_info = forte.make_mo_space_info_from_map(wfn, mo_spaces, [])
    
    # make a ForteIntegral object
    ints = forte.make_forte_integrals(ref_wfn, options, mo_space_info)

    # create active space integrals for CASCI
    as_ints = forte.make_active_space_ints(mo_space_info, ints, "ACTIVE", ["RESTRICTED_DOCC"]);
    
    # SCFInfo object: stores doccpi, orbital energies, etc.
    scf_info = forte.SCFInfo(ref_wfn)
    
    # StateInfo: state irrep, multiplicity, nalpha electron, etc.
    state = forte.make_state_info_from_psi_wfn(ref_wfn)
    
    # build a map {StateInfo: a list of weights} for multi-state computations
    state_weights_map = forte.make_state_weights_map(forte_options, ref_wfn)
    
    # converts {StateInfo: weights} to {StateInfo: nroots}
    state_map = forte.to_state_nroots_map(state_weights_map)
    
    # create an active space solver object and compute the energy
    as_solver_type = 'FCI'  # 'CAS', 'ACI', 'DMRG', 'V2RDM'
    as_solver = forte.make_active_space_solver(as_solver_type, state_map, scf_info,
                                               mo_space_info, as_ints, forte_options)
    state_energies_list = as_solver.compute_energy()  # a map {StateInfo: a list of energies}
    
    # compute averaged energy --- reference energy for DSRG
    Eref = forte.compute_average_state_energy(state_energies_list, state_weights_map)
    
    # compute RDMs
    rdms = as_solver.compute_average_rdms(state_weights_map, rdm_level)
    
    # semicanonicalize orbitals
    semi = forte.SemiCanonical(mo_space_info, ints, forte_options)
    semi.semicanonicalize(rdms, rdm_level)
        
    return Eref, mo_space_info, ints, rdms, as_ints

In [ ]:
mo_spaces = {'FROZEN_DOCC': [1,0,0,0],
             'RESTRICTED_DOCC': [1,0,0,0],
             'ACTIVE': [2,0,1,2]}

Eref, mo_space_info, ints, rdms, as_ints = pre_dsrg(wfn, mo_spaces)
print(f"Forte reference energy:  {Eref:18.12f} Eh")
print(f"CASSCF energy from Psi4: {Ecas:18.12f} Eh")

# Einstein summation convention is assumed throughout this notebook !!!

For example:

\begin{equation}
v^{ab}_{ij} t^{ij}_{ab} \equiv \sum_{ij} \sum_{ab} v^{ab}_{ij} t^{ij}_{ab} = \sum_{ij} \sum_{ab} v^{ij}_{ab} t^{ij}_{ab}
\end{equation}

# Orbital indices conventions

Core orbitals ($\bf C$): $m, n$

Active orbitals ($\bf A$): $u,v,w,x,y,z$

Virtual orbitals ($\bf V$): $e, f$

Generalized holes ($\bf H = C \cup A$): $i,j,k,l$

Generalized particles ($\bf P = A \cup V$): $a,b,c,d$

Any generic orbital: $p, q, r, s$

# Build reference energy using densities, just for fun

The reference energy is given by
\begin{align}
E_\text{ref} = h^{i}_{j} \gamma_{i}^{j} + \frac{1}{4} v^{ij}_{kl} \gamma^{kl}_{ij}.
\end{align}
The densities are labeled by generalized hole indices in the above equation and they are given by
\begin{align}
\gamma^{i}_{j} &= \langle \Psi_0 | \hat{a}^\dagger_i \hat{a}_j | \Psi_0 \rangle, \\
\gamma^{ij}_{kl} &= \langle \Psi_0 | \hat{a}^\dagger_i \hat{a}^\dagger_j \hat{a}_l \hat{a}_k | \Psi_0 \rangle .
\end{align}

We can implement the reference energy using this way.
However, the RDMs from ActiveSpaceSolver are active-space only, that is, we only have the following active-space densities
\begin{align}
\gamma^{u}_{v} &= \langle \Psi_0 | \hat{a}^\dagger_u \hat{a}_v | \Psi_0 \rangle, \\
\gamma^{uv}_{xy} &= \langle \Psi_0 | \hat{a}^\dagger_u \hat{a}^\dagger_v \hat{a}_y \hat{a}_x | \Psi_0 \rangle .
\end{align}

Due to the structure of active-space-based methods, let us simplify the reference energy expression
\begin{align}
E_\text{ref} &= \sum_{m} h^{m}_{m} + h^{u}_{v} \gamma_{u}^{v} + \frac{1}{2} \sum_{mn} v^{mn}_{mn} + \sum_{m} \sum_{uv} v^{mu}_{mv} \gamma^{v}_{u} + \frac{1}{4} v^{uv}_{xy} \gamma^{xy}_{uv} \\
&= \underbrace{\sum_{m} h^{m}_{m} + \frac{1}{2} \sum_{mn} v^{mn}_{mn}}_\text{scalar term} + \underbrace{\left( h^{u}_{v} + \sum_{m} v^{mu}_{mv} \right)}_\text{active one-body integrals} \gamma_{u}^{v} + \frac{1}{4} v^{uv}_{xy} \gamma^{xy}_{uv} .
\end{align}
At this point, we can implement the reference energy in a straightforward way.

In [ ]:
import numpy as np

In [ ]:
# TODO: figure out core, active and virtual MO indices
core_mos = None
actv_mos = None
virt_mos = None

mos = {'c': core_mos, 'a': actv_mos, 'v': virt_mos}
nmos = {k: len(v) for k, v in mos.items()}

nc, na, nv = nmos['c'], nmos['a'], nmos['v']

print(f"Core MOs: {mos['c']}; size: {nmos['c']}")
print(f"Active MOs: {mos['a']}; size: {nmos['a']}")
print(f"Virtual MOs: {mos['v']}; size: {nmos['v']}")

In [ ]:
# TODO: get densities
D1 = {'a': None,
      'b': None}
D2 = {'aa': None,
      'ab': None,
      'bb': None}

In [ ]:
# TODO: grab 1e-itegrals
oei = {'a': {}, 'b': {}}
for block in ['cc', 'ca', 'cv', 'aa', 'av', 'vv']:
    pass

In [ ]:
# TODO: compute oei core contributions to the reference energy
Eref_ele1 = 0.0
Eref_ele1

In [ ]:
# compute tei core-core contributions to the reference energy
Eref_ele2 = 0.0



Eref_ele2

In [ ]:
# compute effective oei active contributions to the reference energy
Eref_ele3 = 0.0



Eref_ele3

In [ ]:
# compute tei active-active contributions to the reference energy
Eref_ele4 = 0.0



Eref_ele4

In [ ]:
# total electronic energy
Eref_ele = Eref_ele1 + Eref_ele2 + Eref_ele3 + Eref_ele4

# total energy
Eref_hand = Eref_ele + as_ints.nuclear_repulsion_energy() + as_ints.frozen_core_energy()

print(f"Reference energy computed by hand:  {Eref_hand}")
print(f"Reference energy computed by Forte: {Eref}")
assert(abs(Eref - Eref_hand) < 1.0e-10)

# Build Fock matrix

The spin-orbital generalized Fock matrix is given by:

\begin{align}
f^p_q = h^p_q + \sum_{m} v^{pm}_{qm} + \sum_{uv} v^{pu}_{qv} \gamma^{v}_{u}.
\end{align}

**What to expect for the structure of Fock matrix?**

In [ ]:
def build_fock(ints, oei, opdm):
    """
    Build generalized Fock matrix.
    :param ints: ForteIntegrals used to grab antisymmetrized 2e-integrals
    :param oei: structrued 1e-integrals (reuse from previous cells) {spin: data}
    :param opdm: structured 1RDM (reuse previous data) {spin: data}
    :return: structured Fock matrix {spin: {block label: data}}
    """
    # initialize with 1e-integrals
    F = {'a': {k: v.copy() for k, v in oei['a'].items()},
         'b': {k: v.copy() for k, v in oei['b'].items()}}
    
    mos = {'c': core_mos, 'a': actv_mos, 'v': virt_mos}
    
    # build alpha spin of Fock
    F['a'] = build_fock_alpha(F['a'], ints, opdm, mos)
    
    # build beta spin of Fock
    F['b'] = build_fock_beta(F['b'], ints, opdm, mos)
    
    return F

In [ ]:
# TODO: build alpha Fock

def build_fock_alpha(fock, ints, opdm, mos):
    """
    Modify alpha spin Fock matrix.
    :param fock: alpha Fock initialized with 1e-integrals with structure {block label: data}
    :param ints: ForteIntegrals object
    :param opdm: structured 1RDM (reuse previous data) {spin: data}
    :param mos: a map of MOs {space label: a list of absolute indices}
    :return: modified alpha Fock
    """
    
    
    return fock

In [ ]:
# TODO: build beta Fock

def build_fock_beta(fock, ints, opdm, mos):
    """
    Modify beta spin Fock matrix.
    :param fock: beta Fock initialized with 1e-integrals with structure {block label: data}
    :param ints: ForteIntegrals object
    :param opdm: structured 1RDM (reuse previous data) {spin: data}
    :param mos: a map of MOs {space label: a list of absolute indices}
    :return: modified beta Fock
    """

    
    return fock

In [ ]:
F = build_fock(ints, oei, D1)

In [ ]:
for block in ['cc', 'ca', 'cv', 'aa', 'av', 'vv']:
    print(f"Block {block}")
    print(F['a'][block])
    assert(np.all(np.abs(F['a'][block] - F['b'][block]) < 1.0e-10))

### Grab orbital energies (diagonal elements of Fock)

We will keep only one copy of orbital energies because alpha orbitals are equivalent to those of beta in this case.

In [ ]:
# grab orbital energies
Fdiag = {'c': np.diag(F['a']['cc']),
         'a': np.diag(F['a']['aa']),
         'v': np.diag(F['a']['vv'])}
Fdiag

# Build cluster amplitudes

The singles and doubles amplitudes are respectively expressed as

\begin{align}
t_{a}^{i} &= \left[ f_{a}^{i} + \sum_{ux} (\epsilon_{x} - \epsilon_{u}) t_{ax}^{iu} \gamma^{x}_{u} \right] \frac{ 1 - e^{-s \left( \Delta_{a}^{i} \right)^2} }{\Delta_{a}^{i}}, \\
t_{ab}^{ij} &= v_{ab}^{ij} \frac{ 1 - e^{-s \left( \Delta_{ab}^{ij} \right)^2} }{\Delta_{ab}^{ij}}.
\end{align}

These expressions suggest that we need to first build doubles amplitudes and then singles.

### Flow parameter

In [ ]:
flow_param = 1.0

### Block labels for T1 and T2

In [ ]:
# T1 blocks
T1blocks = ['cv', 'ca', 'av']

# T2 blocks
T2blocks = ['ccvv', 'ccva', 'ccaa', 'cavv', 'cava', 'caaa', 'aavv', 'aava']

### Regularizer

In [ ]:
# TODO: regularize 2-body data
def regularize_two_body(v, labels, Fdiag, s):
    """
    Regularize data using v * (1 - exp(-s * D^2)) / D when v is a 4-way tensor.
    :param v: a numpy ndarray of dimension 4
    :param labels: a tuple of space names, should be the same size of the shape of v
    :param Fdiag: orbital energies {space name: a list of energies}
    :param s: the flow parameter
    :return: modified 2-body data
    """
    out = v.copy()
    
    
    
    return out

### Compute T2

\begin{align}
t_{ab}^{ij} = v_{ab}^{ij} \frac{ 1 - e^{-s \left( \Delta_{ab}^{ij} \right)^2} }{\Delta_{ab}^{ij}}.
\end{align}

In [ ]:
# TODO: build doubles amplitudes

def build_t2_amplitudes(ints, Fdiag, s):
    """
    Compute T2 amplitudes.
    :param ints: ForteIntegrals object
    :param Fdiag: orbital energies in format {space name: a list of energies}
    :param s: the flow parameter
    :return: T2 amplitudes in format {spin cases: {block label: data}}
    """
    T2 = {'aa': {}, 'ab': {}, 'bb': {}}
    mos = {'c': core_mos, 'a': actv_mos, 'v': virt_mos}
    

    
    return T2

In [ ]:
T2 = build_t2_amplitudes(ints, Fdiag, flow_param)
print(np.linalg.norm(T2['aa']['cavv']))
print(np.linalg.norm(T2['ab']['cavv']))
print(np.linalg.norm(T2['bb']['cavv']))

### Compute T1

\begin{align}
t_{a}^{i} = \left[ f_{a}^{i} + \sum_{ux} (\epsilon_{x} - \epsilon_{u}) t_{ax}^{iu} \gamma^{x}_{u} \right] \frac{ 1 - e^{-s \left( \Delta_{a}^{i} \right)^2} }{\Delta_{a}^{i}}.
\end{align}

In [ ]:
# TODO: form the intermediate of T2 contracting with energy-weighted density

def build_rfock_intermediate(F, T2, opdm, Fdiag):
    """
    Build an intermediate of Fock + T2 contracting with energy-weighted density.
    :param F: Fock matrix in format {spin cases: {block labels: data}}
    :param T2: T2 amplitudes in format {spin cases: {block labels: data}}
    :param opdm: 1RDM in format {spin cases: data}
    :param Fdiag: orbital energies {space name: data}
    :return: the intermediate f + t2 * ew_d
    """
    iF = {'a': {k: F['a'][k].copy() for k in T1blocks},
          'b': {k: F['b'][k].copy() for k in T1blocks}}
    

    
    return iF

In [ ]:
# TODO: regularize 1-body data
def regularize_one_body(v, labels, Fdiag, s):
    """
    Regularize data using v * (1 - exp(-s * D^2)) / D when v is a 2-way tensor.
    :param v: a numpy ndarray of dimension 2
    :param labels: a tuple of space names, should be the same size of the shape of v
    :param Fdiag: orbital energies {space name: a list of energies}
    :param s: the flow parameter
    :return: modified 1-body data
    """
    out = v.copy()

    
    
    return out

In [ ]:
# TODO: build singles amplitudes

def build_t1_amplitudes(iF, T2, opdm, Fdiag, s):
    """
    Compute T1 amplitudes.
    :param iF: modified Fock matrix in format {spin cases: {block label: data}}
    :param T2: doubles amplitudes {spin cases: {block label: data}}
    :param opdm: 1RDM in format {spin cases: data}
    :param Fdiag: orbital energies in format {space name: a list of energies}
    :param s: the flow parameter
    :return: T1 amplitudes in format {spin cases: {block label: data}}
    """
    T1 = {'a': {}, 'b': {}}
    mos = {'c': core_mos, 'a': actv_mos, 'v': virt_mos}
    

    
    return T1

In [ ]:
iF = build_rfock_intermediate(F, T2, D1, Fdiag)
T1 = build_t1_amplitudes(iF, T2, D1, Fdiag, flow_param)

# Renormalize one- and two-electron integrals

The spin-orbital expression is given by

\begin{align}
\tilde{f}^i_a &= f^i_a \left[ 1 + e^{-s (\Delta^{i}_{a})^2} \right] + e^{-s (\Delta^{i}_{a})^2} \sum_{ux} (\epsilon_{x} - \epsilon_{u}) t_{ax}^{iu} \gamma^{x}_{u} , \\
\tilde{v}_{ab}^{ij} &= v_{ab}^{ij} \left[ 1 + e^{-s(\Delta_{ab}^{ij})^2} \right] .
\end{align}